In [1]:
#保存するディレクトリ
dir = 'takamatsu/horsepage/2023'

In [2]:
#horseのURL
HORSE_URL = ['https://db.netkeiba.com/horse/2015104364',
            'https://db.netkeiba.com/horse/2019102788',
            'https://db.netkeiba.com/horse/2015101028',
            'https://db.netkeiba.com/horse/2018105610',
            'https://db.netkeiba.com/horse/2018105069',
            'https://db.netkeiba.com/horse/2016106453',
            'https://db.netkeiba.com/horse/2017100434',
            'https://db.netkeiba.com/horse/2017110144',
            'https://db.netkeiba.com/horse/2017100583',
            'https://db.netkeiba.com/horse/2017100381',
            'https://db.netkeiba.com/horse/2018105204',
            'https://db.netkeiba.com/horse/2019105126',
            'https://db.netkeiba.com/horse/2016102197',
            'https://db.netkeiba.com/horse/2019106526',
            'https://db.netkeiba.com/horse/2019103753',
            'https://db.netkeiba.com/horse/2018104934',
            'https://db.netkeiba.com/horse/2017101121',
            'https://db.netkeiba.com/horse/2019100213'
            ]

予測したいレースに関連する馬をスクレイピングするコード

In [3]:
#インポート文
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

import os
import time
import requests

import re

In [4]:
HTML_DIR = "html/"+dir
if not os.path.isdir(HTML_DIR):
    os.makedirs(HTML_DIR)
        
for url in HORSE_URL:
    list = url.split("/")
    horse_id = list[-1]
    save_file_path = HTML_DIR+"/"+horse_id+'.html'
    response = requests.get(url)
    response.encoding = response.apparent_encoding
    html = response.text
    time.sleep(5)
    with open(save_file_path, 'w') as file:
        file.write(html)

In [5]:
horse_data_info_columns=[
    'horse_id',
    'bday',
    'tame_id',
    'owner_id',
    'producer_id',
    'production area',
    'auction price',
    'winnings',
    'lifetime record',
    'wined race title',
    'inbreeding-1',
    'inbreeding-2',
    'father',
    'faths father',
    'faths mother',
    'mother',
    'moths father',
    'moths mother'
]

In [6]:
horse_race_columns=["date",
                    "whereracecourse",
                    "weather",
                    "race_round",
                    "race_title",
                    "race_id",
                    "total_horse_number",
                    "frame_number",
                    "horse_number",
                    "odds",
                    "popular",
                    "rank",
                    "rider_id",
                    "horse_wight",
                    "distance",
                    "groud_status",
                    'goal_time',
                    'goal_time_dif',
                    'half_way_rank',
                    'pace',
                    'last_time',
                    'horse_weight',
                    'runner-up-horse-id',
                    'prize'
                    ]

In [7]:
def get_horse_data_by_html(horse_id, html):
    horse_list = [horse_id]
    horse_race_list = []
    soup = BeautifulSoup(html, 'html.parser')
    horse_tables = soup.find("table", class_="db_prof_table")
    horse_table = horse_tables.findAll("td")
    #for i in range(10):
    horse_list.append(horse_table[0].get_text())
    horse_list.append(horse_table[1].find('a').get('href').split("/")[-2])
    horse_list.append(horse_table[2].find('a').get('href').split("/")[-2])
    horse_list.append(horse_table[3].find('a').get('href').split("/")[-2])
    horse_list.append(horse_table[4].get_text())
    horse_list.append(horse_table[5].get_text())
    horse_list.append(horse_table[6].get_text())
    horse_list.append(horse_table[7].get_text())
    horse_list.append(horse_table[8].find('a').get('href').split("/")[-2])
    
    # horse_table[8]からすべてのリンクのURLの一部を取得する
    url_parts = [link.get('href').split("/")[-2] for link in horse_table[9].find_all('a')]

    # url_partsの要素数が2未満の場合、残りの要素をNaNで埋める
    while len(url_parts) < 2:
        url_parts.append(np.nan)

    # horse_listにurl_partsを追加する
    horse_list.extend(url_parts)
    
    #horse_list.append(horse_table[9].get_text())
    blood_tables = soup.find("table", class_="blood_table")
    blood_table = blood_tables.findAll("td")
    #for i in range(6):
    horse_list.append(blood_table[0].find('a').get('href').split("/")[-2])
    horse_list.append(blood_table[1].find('a').get('href').split("/")[-2])
    horse_list.append(blood_table[2].find('a').get('href').split("/")[-2])
    horse_list.append(blood_table[3].find('a').get('href').split("/")[-2])
    horse_list.append(blood_table[4].find('a').get('href').split("/")[-2])
    horse_list.append(blood_table[5].find('a').get('href').split("/")[-2])    

    
    
#horse_race
    horse_tables = soup.find("table", class_="db_h_race_results")
    horse_trs = horse_tables.findAll("tr")
    for index, horse_tr in enumerate(horse_trs):
        if index == 0:
            continue
        horse_tds = horse_tr.findAll("td")
        if horse_tds[26].find('a') is not None:
            horse_race_list.append([horse_tds[0].get_text(),
                                    horse_tds[1].get_text(),
                                    horse_tds[2].get_text(),
                                    horse_tds[3].get_text(),
                                    horse_tds[4].get_text(),
                                    horse_tds[4].find('a').get('href').split("/")[-2],
                                    horse_tds[6].get_text(),
                                    horse_tds[7].get_text(),
                                    horse_tds[8].get_text(),
                                    horse_tds[9].get_text(),
                                    horse_tds[10].get_text(),
                                    horse_tds[11].get_text(),
                                    horse_tds[12].find('a').get('href').split("/")[-2],
                                    horse_tds[13].get_text(),
                                    horse_tds[14].get_text(),
                                    horse_tds[15].get_text(),
                                    horse_tds[17].get_text(),
                                    horse_tds[18].get_text(),
                                    horse_tds[20].get_text(),
                                    horse_tds[21].get_text(),
                                    horse_tds[22].get_text(),
                                    horse_tds[23].get_text(),
                                    horse_tds[26].find('a').get('href').split("/")[-2],
                                    horse_tds[27].get_text(),
                                   ])
        else:
            horse_race_list.append([horse_tds[0].get_text(),
                                    horse_tds[1].get_text(),
                                    horse_tds[2].get_text(),
                                    horse_tds[3].get_text(),
                                    horse_tds[4].get_text(),
                                    horse_tds[4].find('a').get('href').split("/")[-2],
                                    horse_tds[6].get_text(),
                                    horse_tds[7].get_text(),
                                    horse_tds[8].get_text(),
                                    horse_tds[9].get_text(),
                                    horse_tds[10].get_text(),
                                    horse_tds[11].get_text(),
                                    horse_tds[12].find('a').get('href').split("/")[-2],
                                    horse_tds[13].get_text(),
                                    horse_tds[14].get_text(),
                                    horse_tds[15].get_text(),
                                    horse_tds[17].get_text(),
                                    horse_tds[18].get_text(),
                                    horse_tds[20].get_text(),
                                    horse_tds[21].get_text(),
                                    horse_tds[22].get_text(),
                                    horse_tds[23].get_text(),
                                    None,
                                    horse_tds[27].get_text(),
                                   ])
                                        
    horse_race_tmp_df = pd.DataFrame(horse_race_list, columns=horse_race_columns)
    horse_race_tmp_df.loc[:, 'horse_id'] = horse_id
    
    return horse_list , horse_race_tmp_df

In [8]:
CSV_DIR = "csv/"+dir
if not os.path.isdir(CSV_DIR):
    os.makedirs(CSV_DIR)
horse_info_csv = CSV_DIR+"/horse-info.csv"
horse_race_csv = CSV_DIR+"/horse-race.csv"
horse_info_df = pd.DataFrame(columns=horse_data_info_columns)
horse_race_df = pd.DataFrame()
if os.path.isdir(HTML_DIR):
    file_list = os.listdir(HTML_DIR)
    for file_name in file_list:
        with open(HTML_DIR+"/"+file_name, "r") as f:
            html = f.read()
            list = file_name.split(".")
            horse_id = list[-2]
            horse_list , horse_race_tmp_df = get_horse_data_by_html(horse_id, html) 
            horse_se = pd.Series(horse_list, index=horse_info_df.columns)
            horse_info_df = pd.concat([horse_info_df, horse_se.to_frame().T], ignore_index=True)
            horse_race_df = pd.concat([horse_race_df, horse_race_tmp_df], axis=0, ignore_index=True)       

### cleaning

##### horse_race

In [9]:
horse_info_df.head(1)

,horse_id,bday,tame_id,owner_id,producer_id,production area,auction price,winnings,lifetime record,wined race title,inbreeding-1,inbreeding-2,father,faths father,faths mother,mother,moths father,moths mother
0,2015101028,2015年3月3日,01027,865800,103341,日高町,\n-\n,"\n\n\n1億7,113万円 (中央)\n\n",28戦6勝 [6-7-3-12],202105030811,2011100025,2019101266,2008103552,2001103460,1996107173,2003105704,1989108341,000a00682c


##### date

In [10]:
horse_race_df.head(1)

,date,whereracecourse,weather,race_round,race_title,race_id,total_horse_number,frame_number,horse_number,odds,...,groud_status,goal_time,goal_time_dif,half_way_rank,pace,last_time,horse_weight,runner-up-horse-id,prize,horse_id
0,2023/01/29,1中京10,晴,11,シルクロードS(G3),202307011011,15,7,13,9.9,...,良,1:08.8,1.5,3-3,33.8-33.5,34.7,508(+2),2019103753,,2015101028


In [11]:
horse_race_df['date'] = pd.to_datetime(horse_race_df['date'], format='%Y/%m/%d')

##### race_title

In [12]:
# 'race_title' 列から () の中身を抽出する
horse_race_df['race_type'] = horse_race_df['race_title'].str.extract('\((.*?)\)')

# 'race_type' 列の欠損値を空文字列に置き換える
horse_race_df['race_type'].fillna('', inplace=True)

##### odds

In [13]:
#数字以外の文字を削除
horse_race_df['odds'] = horse_race_df['odds'].apply(lambda x: re.sub('[^0-9\.]', '', x))
#空白を削除
horse_race_df['odds'] = horse_race_df['odds'].replace('', np.nan)
#欠損値の行を削除
horse_race_df = horse_race_df.dropna(subset=['odds'])

In [14]:
horse_race_df['odds'] = horse_race_df['odds'].astype(float)

##### rank

In [15]:
#数字以外の文字を含む列を削除
horse_race_df = horse_race_df[pd.to_numeric(horse_race_df['rank'], errors='coerce').notnull()]
#数値に変換
horse_race_df['rank'] = horse_race_df['rank'].astype(int)

##### distance

In [16]:
#文字と数値に分離させる
horse_race_df[['surface', 'distance']] = horse_race_df['distance'].str.extract('(\D+)(\d+)')
# 'distance' 列を整数値に変換する
horse_race_df['distance'] = horse_race_df['distance'].astype(int)

##### goal_time

In [17]:
# 正規表現に一致する文字列を削除して、コロンだけを残す
horse_race_df['goal_time'] = horse_race_df['goal_time'].apply(lambda x: re.sub(r"[^0-9:]", "", x))
#空白を削除
horse_race_df['goal_time'] = horse_race_df['goal_time'].replace('', np.nan)
#欠損値の行を削除
horse_race_df = horse_race_df.dropna(subset=['goal_time'])
#数値に変換
def to_seconds(time_str):
    minutes, seconds = map(float, time_str.split(':'))
    return minutes * 60 + seconds

horse_race_df['goal_time'] = horse_race_df['goal_time'].apply(to_seconds)

##### goal_time_dif

In [18]:
horse_race_df['goal_time_dif'] = horse_race_df['goal_time_dif'].astype(float)

In [19]:
horse_race_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 344 entries, 0 to 349
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                344 non-null    datetime64[ns]
 1   whereracecourse     344 non-null    object        
 2   weather             344 non-null    object        
 3   race_round          344 non-null    object        
 4   race_title          344 non-null    object        
 5   race_id             344 non-null    object        
 6   total_horse_number  344 non-null    object        
 7   frame_number        344 non-null    object        
 8   horse_number        344 non-null    object        
 9   odds                344 non-null    float64       
 10  popular             344 non-null    object        
 11  rank                344 non-null    int32         
 12  rider_id            344 non-null    object        
 13  horse_wight         344 non-null    object        

In [20]:
horse_info_df.to_csv(horse_info_csv, header=True, index=False)
horse_race_df.to_csv(horse_race_csv, header=True, index=False)

##### 以下メモ

In [21]:
# horse_info_df['horse_id'] = horse_info_df['horse_id']
# horse_info_df['tame_id'] = horse_info_df['tame_id']
# horse_info_df['owner_id'] = horse_info_df['owner_id']

#以下文字列の可能性あり
# horse_info_df['inbreeding-1'] = horse_info_df['inbreeding-1']
# horse_info_df['inbreeding-2'] = horse_info_df['inbreeding-2']
# horse_info_df['father'] = horse_info_df['father']
# horse_info_df['mother'] = horse_info_df['mother']

In [22]:
# # データを読み込む
# data = pd.read_csv('data.csv')

# # 日付でソートする
# data = data.sort_values('date')

# # 同じ ID を持つデータをグループ化する
# grouped_data = data.groupby('ID')

# # 各グループの最初のデータを選択する
# latest_data = grouped_data.last()

In [23]:
# horse_list = [horse_id]
# horse_race_list = []
# soup = BeautifulSoup(html, 'html.parser')
# horse_tables = soup.find("table", class_="db_h_race_results")
# horse_trs = horse_tables.findAll("tr")
# for index, horse_tr in enumerate(horse_trs):
#     if index == 0:
#         continue
#     horse_tds = horse_tr.findAll("td")
#     horse_race_list.append([horse_tds[0].get_text(),
#                             horse_tds[1].get_text(),
#                             horse_tds[2].get_text(),
#                             horse_tds[3].get_text(),
#                             horse_tds[4].find('a').get('href').split("/")[-2],
#                             horse_tds[6].get_text(),
#                             horse_tds[7].get_text(),
#                             horse_tds[8].get_text(),
#                             horse_tds[9].get_text(),
#                             horse_tds[10].get_text(),
#                             horse_tds[11].get_text(),
#                             horse_tds[12].find('a').get('href').split("/")[-2],
#                             horse_tds[13].get_text(),
#                             horse_tds[14].get_text(),
#                             horse_tds[15].get_text(),
#                             horse_tds[17].get_text(),
#                             horse_tds[18].get_text(),
#                             horse_tds[20].get_text(),
#                             horse_tds[21].get_text(),
#                             horse_tds[22].get_text(),
#                             horse_tds[23].get_text(),
#                             horse_tds[26].find('a').get('href').split("/")[-2],
#                             horse_tds[27].get_text(),
#                            ])

# df = pd.DataFrame(horse_race_list, columns=horse_race_columns)
# df.loc[:, 'horse_id'] = horse_id